In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Create a csv file for all the historical data of all the related stocks

for element in Competitors:
  df = yf.download(element)
  df.reset_index(inplace=True)
  df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')
  df.to_csv(element + 'output.csv', index=False)

for element in RelatedListNonIndex:
  df = yf.download(element)
  df.reset_index(inplace=True)
  df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')
  df.to_csv(element + 'output.csv', index=False)

for element in RelatedListIndex:
  df = yf.download(element)
  df.reset_index(inplace=True)
  df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')
  df.to_csv(element + 'output.csv', index=False)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

ERROR 
1 Failed download:
ERROR:yfinance:
1 Failed download:
ERROR ['NASDAQ']: Exception('NASDAQ: 1d data not available for startTime=-2208994789 and endTime=1687721949. Only 100 years worth of day granularity data are allowed to be fetched per request.')
ERROR:yfinance:['NASDAQ']: Exception('NASDAQ: 1d data not available for startTime=-2208994789 and endTime=1687721949. Only 100 years worth of day granularity data are allowed to be fetched per request.')


In [ ]:
from pandas_datareader import data
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import urllib.request, json
import os
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import math
import numpy as np
import pandas as pd
from datetime import date, timedelta, datetime
from pandas.plotting import register_matplotlib_converters
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import tensorflow as tf
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import RobustScaler, MinMaxScaler
import seaborn as sns

Companies | Source
--- | ----
KLA-Tencor, Tokyo Electron, Lam Research, and Arrow Electronics. | [Forbes](https://www.forbes.com/sites/greatspeculations/2019/08/30/understanding-applied-materials-business-model-and-performance-across-segments/?sh=1f41943276f4)
Intel | [AMAT](https://www.appliedmaterials.com/us/en/blog/blog-posts/applied-materials-earns-intels-2022-epic-outstanding-supplier-award.html)

In [ ]:
#Define company list

Competitors = ["KLAC", "TOELY", "LRCX", "ARW"]   # similar companies
RelatedListNonIndex = ["TSM","INTC", "NVDA", "AMD", "MU"]   # companies Applied Materials supplies to
RelatedListIndex = ["SOXX", "NASDAQ"] # aggregates: semi-conductors, tech

In [ ]:
#create a new file

def create_file(name):
  file = open(name, "w")  # Open file in write mode
  file.write("")
  file.close()

In [ ]:
#read text from a file

def read_file(name):
  file = open(name, "r")  # Open file in read mode
  content = file.read()  # Read the entire content of the file
  file.close()
  return content

In [ ]:
#append text to a file

def append_file(name, text):
  file = open(name, "a")  # Open file in append mode
  file.writelines(','.join(text))  # Append content to the file
  file.write("\n")
  file.close()

In [ ]:
#Update the peratio file
create_file("PEratio.csv")

def update_peratio():
  dayList = []
  for element in Competitors:
    tick = yf.Ticker(element)
    stockDataList = []
    stockDataList.append(str(datetime.date.today()))
    stockDataList.append(element)
    stockDataList.append("No")
    if 'trailingPE' in tick.info:
      stockDataList.append(str(tick.info["trailingPE"]))
    else:
      stockDataList.append("N/A")
    if 'forwardPE' in tick.info:
      stockDataList.append(str(tick.info["forwardPE"]))
    else:
      stockDataList.append("N/A")
    dayList.append(stockDataList)

  for element in RelatedListNonIndex:
    tick = yf.Ticker(element)
    stockDataList = []
    stockDataList.append(str(datetime.date.today()))
    stockDataList.append(element)
    stockDataList.append("No")
    if 'trailingPE' in tick.info:
      stockDataList.append(str(tick.info["trailingPE"]))
    else:
      stockDataList.append("N/A")
    if 'forwardPE' in tick.info:
      stockDataList.append(str(tick.info["forwardPE"]))
    else:
      stockDataList.append("N/A")
    dayList.append(stockDataList)

  for element in RelatedListIndex:
    tick = yf.Ticker(element)
    stockDataList = []
    stockDataList.append(str(datetime.date.today()))
    stockDataList.append(element)
    stockDataList.append("Yes")
    if 'trailingPE' in tick.info:
      stockDataList.append(str(tick.info["trailingPE"]))
    else:
      stockDataList.append("N/A")
    if 'forwardPE' in tick.info:
      stockDataList.append(str(tick.info["forwardPE"]))
    else:
      stockDataList.append("N/A")
    dayList.append(stockDataList)
  return dayList

labels = ["Date", "Ticker", "Is Index", "TrailingPE", "ForwardPE"]
append_file("PEratio.csv", labels)
dataFrame = update_peratio()
for i in range (len(dataFrame)):
  append_file("PEratio.csv", dataFrame[i])

In [ ]:
start_date = dt.datetime.now() - dt.timedelta(days=365 * 13+170)
end_date = dt.datetime.now()

for element in Competitors:

  df = yf.download(element, start=start_date, end=end_date)

  df.reset_index(inplace=True)

  df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')
  df['50_day_ma'] = df['Close'].rolling(window=50).mean()
  df['200_day_ma'] = df['Close'].rolling(window=200).mean()
  df['10_day_vol_avg'] = df['Volume'].rolling(window=10).mean()
  df['90_day_vol_avg'] = df['Volume'].rolling(window=90).mean()

  df = df[['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', '50_day_ma', '200_day_ma', '10_day_vol_avg', '90_day_vol_avg']]

  print(df)

df.to_csv('AMAToutput.csv', index=False)

print(df)

[*********************100%***********************]  1 of 1 completed
           Date        Open        High         Low       Close   Adj Close  \
0    2010-01-13   35.889999   36.270000   34.950001   36.090000   20.883665   
1    2010-01-14   35.730000   35.950001   34.810001   35.000000   20.252934   
2    2010-01-15   34.930000   35.240002   33.270000   33.340000   19.292366   
3    2010-01-19   33.480000   33.770000   33.200001   33.639999   19.465969   
4    2010-01-20   33.299999   33.340000   32.619999   33.230000   19.228718   
...         ...         ...         ...         ...         ...         ...   
3382 2023-06-22  453.049988  467.890015  452.959991  467.380005  467.380005   
3383 2023-06-23  457.500000  459.950012  454.589996  456.950012  456.950012   
3384 2023-06-26  458.100006  468.350006  458.100006  458.549988  458.549988   
3385 2023-06-27  460.220001  478.630005  460.220001  476.929993  476.929993   
3386 2023-06-28  470.410004  477.559998  469.519989  474.42999